### <bold>Projet prix des carburants en France entre 2013 et 2023<bold>

1.	Création des tables

In [1]:
import sqlite3

# Variables
nom_database = 'C:\\Users\\amala\\Desktop\\ProjetsData\\PowerBI\\PrixCarburantFrance\\data\\db_CarburantsFr.db'

table_principale = 'fait_carburants'
table_secondaire_1 = 'dim_carburants'
table_secondaire_2 = 'dim_geoloc'

connexion = sqlite3.connect(nom_database)
curseur = connexion.cursor()

# Creation table principale
creation_table_prix_moyen_carburants = f"""
    CREATE TABLE IF NOT EXISTS {table_principale}(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        code_carburant INTEGER,
        date DATE,
        dpt INTEGER,
        nb_stations INTEGER,
        prix_moyen FLOAT,
        station_type TEXT,
        FOREIGN KEY(code_carburant) REFERENCES {table_secondaire_1}(id),
        FOREIGN KEY(dpt) REFERENCES {table_secondaire_2}(code_departement)
    )
"""
curseur.execute(creation_table_prix_moyen_carburants)

# Creation table secondaire "types_carburant"
creation_table_types_carburant = f"""
    CREATE TABLE IF NOT EXISTS {table_secondaire_1}(
        id_carburant INTEGER PRIMARY KEY,
        ancienne_denomination TEXT NOT NULL UNIQUE,
        nom_carburant TEXT NOT NULL UNIQUE,
        nature_carburant TEXT NOT NULL UNIQUE
    )
"""
curseur.execute(creation_table_types_carburant)

# TABLE "geoloc_stations"
creation_table_geoloc_stations = f"""
    CREATE TABLE IF NOT EXISTS {table_secondaire_2}(
        code_departement INTEGER PRIMARY KEY,
        nom_departement TEXT NOT NULL UNIQUE,
        nom_region TEXT NOT NULL UNIQUE 
    )
"""
curseur.execute(creation_table_geoloc_stations)

connexion.commit()
connexion.close()

2.	Récupération des données d'intérêt des fichiers XML de 2013 à 2023 et création d'un fichier csv unique pour import dans Power BI

In [1]:
# code retenu pour visualisation ==> group by

import xml.etree.ElementTree as ET
import pandas as pd

annees = ['2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']

# Créer une liste vide pour stocker les données de toutes les années
data = []


for annee in annees:
    fichier_xml = f"C:\\Users\\amala\\Desktop\\ProjetsData\\PowerBI\\PrixCarburantFrance\\data\\PrixCarburants_annuel_{annee}.xml"

    tree = ET.parse(fichier_xml)
    root = tree.getroot()

    # Extraire les données des éléments pdv
    for pdv_element in root.findall('.//pdv'):
        id_station = pdv_element.get('id', '')
        cp = pdv_element.get('cp', '')

        # Parcourir tous les éléments prix à l'intérieur de l'élément pdv
        for prix_element in pdv_element.findall('.//prix'):
            prix = prix_element.attrib.get('valeur', '')
            id_carb = prix_element.attrib.get('id', '')
            maj = prix_element.attrib.get('maj', '')

            # Nettoyage de la valeur prix
            if prix.strip():
                if int(annee) < 2022:
                    prix = str(float(prix) / 1000)
            

            # Ajouter les données à la liste
            data.append({
                'id_station': id_station,
                'cp': cp,
                'prix' : prix,
                'id_carb': id_carb,
                'date': maj
            })

# Créer un DataFrame à partir des données extraites
df_combined = pd.DataFrame(data)

df_combined

# # Nettoyage de la colonne 'cp'
df_combined['cp'] = df_combined['cp'].astype(str)  # Convertir toutes les valeurs en chaînes de caractères
df_combined['cp'] = df_combined['cp'].apply(lambda x: x.zfill(5) if len(x) == 4 else x)
df_combined['dpt'] = df_combined['cp'].apply(lambda x: int(x[:2]))
df_combined = df_combined.drop('cp', axis=1)

# # Filtrer les lignes où 'id_carb' est différent de 0
df_combined = df_combined[df_combined['id_carb'] != '']

# # Nettoyage de la colonne 'date'
df_combined['date'] = df_combined['date'].str.replace('-', '/').str[:10]
df_combined['date'] = pd.to_datetime(df_combined['date'], format='%Y/%m/%d', errors='coerce')

# # Création de la colonne 'annee'
df_combined['annee'] = df_combined['date'].dt.year

# # Nettoyage de la colonne 'prix'
df_combined['prix'] = pd.to_numeric(df_combined['prix'], errors='coerce')

# # Calcul de la moyenne et de l'écart-type de la colonne `prix`
mean_price = df_combined['prix'].mean()
std_price = df_combined['prix'].std()

# # Définition des seuils supérieur et inférieur
lower_threshold = mean_price - 4 * std_price
upper_threshold = mean_price + 4 * std_price

# # Remplacement des valeurs aberrantes dans la colonne `prix` par la moyenne
df_combined['prix'] = df_combined['prix'].apply(lambda x: mean_price if x < lower_threshold or x > upper_threshold else x)

# Groupby avec nomb de sta  tions et prix moyen
# df_grouped = df_combined.groupby(['date', 'dpt']).agg({'id_station': 'nunique'}).reset_index()

df_grouped = df_combined.groupby(['date', 'dpt', 'id_carb']).agg({'id_station': 'first', 'prix': 'mean'}).reset_index()
        
# # Obtenez la première et la dernière année numériquement
min_year = int(min(annees))
max_year = int(max(annees))

csv_filename = f"C:\\Users\\amala\\Desktop\\ProjetsData\\PowerBI\\PrixCarburantFrance\\data\\fait_carburants_{min_year}_{max_year}.csv"
df_grouped.to_csv(csv_filename, index=False)

df_grouped

C:\Users\amala\AppData\Local\Temp\ipykernel_42000\49909646.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,date,dpt,id_carb,id_station,prix
0,2013-01-01,1,1,1000008,1.374500
1,2013-01-01,1,2,1000008,1.512500
2,2013-01-01,1,5,1000008,1.494500
3,2013-01-01,2,1,2000007,1.333750
4,2013-01-01,2,5,2000007,1.472250
...,...,...,...,...,...
2096242,2023-12-31,95,2,95150002,1.800500
2096243,2023-12-31,95,3,95100011,0.958444
2096244,2023-12-31,95,4,95270006,0.987000
2096245,2023-12-31,95,5,95100011,1.801588


In [2]:
# code pour toutes les lignes sans group by

import xml.etree.ElementTree as ET
import pandas as pd

annees = ['2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']

# Créer une liste vide pour stocker les données de toutes les années
data = []


for annee in annees:
    fichier_xml = f"C:\\Users\\amala\\Desktop\\ProjetsData\\PowerBI\\PrixCarburantFrance\\data\\PrixCarburants_annuel_{annee}.xml"

    tree = ET.parse(fichier_xml)
    root = tree.getroot()


    # Extraire les données des éléments pdv
    for pdv_element in root.findall('.//pdv'):
        id_station = pdv_element.get('id', '')
        cp = pdv_element.get('cp', '')
        pop = pdv_element.get('pop', '')  

        # Parcourir tous les éléments prix à l'intérieur de l'élément pdv
        for prix_element in pdv_element.findall('.//prix'):
            prix = prix_element.attrib.get('valeur', '')
            id_carb = prix_element.attrib.get('id', '')
            maj = prix_element.attrib.get('maj', '')

            # Nettoyage de la valeur prix
            if prix.strip():
                if int(annee) < 2022:
                    prix = str(float(prix) / 1000)
            

            # Ajouter les données à la liste
            data.append({
                'id_station': id_station,
                'cp': cp,
                'pop': pop,
                'prix' : prix,
                'id_carb': id_carb,
                'date': maj
            })


# Créer un DataFrame à partir des données extraites
df_combined = pd.DataFrame(data)

df_combined


# # Obtenez la première et la dernière année numériquement
min_year = int(min(annees))
max_year = int(max(annees))

csv_filename = f"C:\\Users\\amala\\Desktop\\ProjetsData\\PowerBI\\PrixCarburantFrance\\data\\fait_carburants_pop_{min_year}_{max_year}.csv"
df_combined.to_csv(csv_filename, index=False)

df_combined

,id_station,cp,pop,prix,id_carb,date
0,1000001,01000,R,1.299,1,2013-01-03 09:17:19.051536
1,1000001,01000,R,1.309,1,2013-01-04 09:23:27.936938
2,1000001,01000,R,1.309,1,2013-01-05 09:25:49.91923
3,1000001,01000,R,1.315,1,2013-01-07 09:30:56.632611
4,1000001,01000,R,1.315,1,2013-01-09 09:53:43.848735
...,...,...,...,...,...,...
41203177,95870010,95870,R,1.989,6,2023-12-12T19:23:23
41203178,95870010,95870,R,1.989,6,2023-12-12T19:40:46
41203179,95870010,95870,R,1.99,6,2023-12-18T20:08:39
41203180,95870010,95870,R,1.99,6,2023-12-30T13:24:39


3.	Remplissage des tables

In [1]:
import sqlite3
import pandas as pd

def inserer_donnees_table(connexion, table_name, csv_file):
    df = pd.read_csv(csv_file)
    df.to_sql(table_name, connexion, if_exists='replace', index=False)

# Variables
nom_database = 'C:\\Users\\amala\\Desktop\\ProjetsData\\PowerBI\\PrixCarburantFrance\\data\\db_CarburantsFr.db'
tables_et_fichiers_csv = [
    ('fait_carburants', 'C:\\Users\\amala\\Desktop\\ProjetsData\\PowerBI\\PrixCarburantFrance\\data\\fait_carburants_2013_2023.csv'),
    ('dim_carburants', 'C:\\Users\\amala\\Desktop\\ProjetsData\\PowerBI\\PrixCarburantFrance\\data\\dim_carburants.csv'),
    ('dim_geoloc', 'C:\\Users\\amala\\Desktop\\ProjetsData\\PowerBI\\PrixCarburantFrance\\data\\dim_geoloc.csv')
                         ]

# Connexion à la base de données
connexion = sqlite3.connect(nom_database)
curseur = connexion.cursor()

# Insertion des données pour chaque table et fichier CSV
for table_name, fichier_csv in tables_et_fichiers_csv:
    inserer_donnees_table(connexion, table_name, fichier_csv)

# Fermeture de la connexion
connexion.close()